In [120]:
import numpy as np
import pandas as pd
import scipy as sp
import resampy as rs 
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.tsa as tsa 
import statsmodels.formula.api as smf
import os
import glob
import random
import string

In [121]:
study_directory = '/Users/alexastefanko/lab/FoGdetection/Subjects'
output_directory = '/Users/alexastefanko/lab/FoGdetection/Output/'
sample_rate = 128

r_acc_ap_col = "R_acc"; #Column name for right accelerometer anterior-posterior
l_acc_ap_col = "L_acc"; #Column name for left accelerometer anterior-posterior
r_gyr_ml_col = "R_gyr"; #Column name for right gyroscope mediolateral
l_gyr_ml_col = "L_gyr"; #Column name for left gyroscop mediolateral

In [122]:
class Fog:
    def __init__(self, filename):
        self.filename = filename
        self.very_short_fog = None
        self.short_fog = None
        self.long_fog = None
        self.very_long_fog = None
        self.NN = None
        self.MM = None

In [151]:
def get_csv_files(directory):
    for dir_name, subdir_list, file_list in os.walk(directory):
        return [os.path.join(dir_name, name) for name in file_list if name.endswith(".csv")]

def create_corr_log(corr_data, threshold):
    log = [None] * len(corr_data)
    idx = 0
    for i in corr_data:
        if i < threshold:
            log[idx] = 1
        else:
            log[idx] = 0
        idx = idx + 1
    return log 
        

def get_starter_idx(data):
    r_ml = data[0]
    mean_r = np.mean(r_ml)
    r_for_cov = [x - mean_r for x in r_ml]
    
    l_ml = data[1]
    mean_l = np.mean(l_ml)
    l_for_cov = [x - mean_l for x in l_ml]

    num_points = len(r_ml)
    array = np.array([r_ml, l_ml])
        
    #R and L angular velocities cross-correlation to find a delay between two
    cross_cov = np.correlate(r_for_cov, l_for_cov, "full")
    shifter = 0
    for idx, num in enumerate(cross_cov):
        if num == np.max(cross_cov):
            shifter = idx
    
    starter = round(abs(shifter - (len(cross_cov)/2)))
    return starter
    
def correlation_based_method(starter, filtered_data):
    #Sync R and L after correction for delay
    t_sensor_1 = filtered_data[0][starter:]
    t_sensor_2 = filtered_data[1][:len(filtered_data) - starter]
    correlated_sensor_data = []
    
    #Ensure bouts have sufficient data
    range_intervals = list(range(0, len(t_sensor_1), 200))
    if len(t_sensor_1) > (200*5):
        for i in range(len(range_intervals) -1):
            start = range_intervals[i]
            end = range_intervals[i+1]
            corr = np.corrcoef(t_sensor_1[start:end], t_sensor_2[start:end])
            correlated_sensor_data.append(corr[0][1])
        abs_corr_sensor_data = np.absolute(correlated_sensor_data)
        corr_log = create_corr_log(abs_corr_sensor_data, 0.5)
    else:
        raise ValueError("Error: There is not enough data to calculate FOG. Please ensure each bout is longer than 10 seconds or sample more often")

def fft_method(starter, ):
    return
    

In [124]:
subject_files = get_csv_files(study_directory)
dataframes = [pd.read_csv(file) for file in subject_files]

In [152]:
new_sample_rate = 200

x = 0
fogs = []
for df in dataframes:
    r_acc_ap = df[r_acc_ap_col].values
    l_acc_ap = df[l_acc_ap_col].values
    r_gyr_ml = df[r_gyr_ml_col].values
    l_gyr_ml = df[l_gyr_ml_col].values
    
    #Resample the data to 200 
    r_ml = rs.resample(r_gyr_ml, sample_rate, new_sample_rate)
    l_ml = rs.resample(l_gyr_ml, sample_rate, new_sample_rate)
    
    #apply low-pass butterworth filter
    a,b = sp.signal.butter(4,5/(200/2))
    
    #difference is here
    sensor_data_filtered=sp.signal.filtfilt(a,b,[r_ml, l_ml], padtype = 'odd', padlen=3*(max(len(b),len(a))-1));
    
    #correlation-based method
    starter_idx = get_starter_idx([r_ml, l_ml])
    correlation_based_method(starter_idx, sensor_data_filtered)
    
    #fft-based method
    
    fog = Fog(subject_files[x])
    fogs.append(fog)
    x = x + 1